![Jane Strret Market Prediction](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c9/Jane_Street_Capital_Logo.svg/1200px-Jane_Street_Capital_Logo.svg.png)

**So, In this competition we need to build a trading Model to find out the Maximum return from the stock Exchange. In the train data, each row is corrosponding to a trade opportuninty or action value. If the action value is 1 the it will be taken and if 0 it will pass on.**

* If this notebbok really helps to get some little bit more insights of teh features and data please UPVOTE my work. I will update more on feature Selection.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



import plotly.express as px
from plotly.subplots import make_subplots

pd.set_option('display.max_columns', 500)

Importing the Necesssary Datas

In [ ]:
# setting the paths to variables to access when required
#Taking  a subset of the data to analyze
train = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv")
sample_test = pd.read_csv("/kaggle/input/jane-street-market-prediction/features.csv")

#Shape of teh Datasets
train.shape, sample_test.shape

Understanding teh Column Types and Missing Values Imputation Techniques

In [ ]:
train.info()

---------------------

## Handling Missing Values


In [ ]:
missing_val = pd.DataFrame(train.isna().sum().sort_values(ascending=False)*100/train.shape[0],columns=['missing %'])[:50]
missing_val.style.background_gradient(cmap='Oranges_r')

There are a total 14 features having Missing Values >10%/ let's found out what are the best stratigies to fill up the missing Values.

**Approach 1: Fill in Missing Values with -999 or -9999**

In this particular scenario the missing values are in random as the data is a real-time trading data. So this approch is appropiate for this dataset.

In [ ]:
train.fillna(-999,inplace=True)

**Approach 2: Fill in Missing Values with mean replacement of each column**

In [ ]:
#Defining a mappper function to replace missing values with withs mean.
for i in list(train.columns):
    x = train[i].mean()
    print(i,"    : ",x)
    #mapp[i] = x
    #train[i]=train[i].fillna(x)

In [ ]:
#Lets Define the action in the data

#We are using pi value 1 to take only the Weight!=0. Weight = 0 is only for data completeness. 
train=train[train['weight']!=0]
train['action']=(train['resp']>0)*1

train.action.value_counts()

Target Label is Balanced or Not?

In [ ]:
sns.countplot(train.action)

From the above plot it is visible that the target feature is balanced so we need not to worry  about the unbalances in the data. 

## Distribution of Different Time Horizons and Action 

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 6))
sns.distplot(train['action'], ax=axs[0])
sns.distplot(train['resp'], ax=axs[1])
sns.distplot(train['weight'], ax=axs[2])

weight is completely left skewed whreas resp has a normal distribution. Let's find out the different time horizons distributions and their differences

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20, 6))
sns.distplot(train['resp_1'], ax=axs[0])
sns.distplot(train['resp_2'], ax=axs[1])
sns.distplot(train['resp_3'], ax=axs[2])
sns.distplot(train['resp_4'], ax=axs[3])

#### Let's found out the cummulative returns of 4 different Time Horizons

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

returns = pd.Series(train['resp']).cumsum()

resp_1= pd.Series(train['resp_1']).cumsum()
resp_2= pd.Series(train['resp_2']).cumsum()
resp_3= pd.Series(train['resp_3']).cumsum()
resp_4= pd.Series(train['resp_4']).cumsum()

ax.set_xlabel ("Trade", fontsize=18)
ax.set_title ("Cummulative Return of 4 diffrerent Time Regions", fontsize=20)
returns.plot()
resp_1.plot()
resp_2.plot()
resp_3.plot()
resp_4.plot()
plt.legend()

## Correlation of Different Time Horizons and Action

In [ ]:
corr = train[['resp_1','resp_2','resp_3','resp_4','resp','action']].corr()

mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

fig = plt.figure(figsize = (15, 8))

# plot the data using seaborn
ax = sns.heatmap(corr, 
                 mask = mask, 
                 annot=True,
                 vmax = 0.3, 
                 square = True,  
                 cmap = "viridis")
# set the title for the figure
ax.set_title("Correlation Actions Vs resp, resp_{1,2,3,4}");

## Analyzing the Noisy Features

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
feature_set = train.loc[:,train.columns.str.contains('feature')].columns

#For analyzing part I am using only 20 features distribution. 
for i in feature_set[:20]:
    fig, ax = plt.subplots(figsize=(10, 4))
    feature_ = pd.Series(train[str(i)]).cumsum()
    ax.set_xlabel ("Trade", fontsize=18)
    ax.set_ylabel (f"feature_{i} (cumulative)", fontsize=15);
    feature_.plot(lw=2)

Thanks @Carl for poiniting out this insight.
[https://www.kaggle.com/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance](http://)

From the above plots we can observe the trends of the features whether it is linear in nature or noisy pattern.
*  Considering teh whole data,OneImportant thing to be noticed here, that feature 3,4,5,6,73,75,76,77,79,82 the trade is decreasing where as for others there is a increasing trends(Curvilinear and Linear Patterns).

* We can sort out the Noisy features as a different subset to the model.

In [ ]:
bad_feats = train[["feature_3","feature_4","feature_5","feature_6","feature_73","feature_75","feature_76","feature_77","feature_79","feature_82","resp","action"]]
bad_feats.corr().style.background_gradient(cmap='viridis', low=1, high=0, axis=None).set_precision(2)

* Insights

1. One thing that is very Noticable that with our action(Traget feature) there is really very correlation with these feature swhich has a negative slope with trade.

## Outlier Analysis

In [ ]:
def find_skewed_boundaries(df, variable, distance):

    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)

    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)

    return upper_boundary, lower_boundary

upper_resp,lower_resp = find_skewed_boundaries(train,'resp',1.5)


print('Capping are',lower_resp,upper_resp)

As it is a Trading Dataset These extreme values may be a important factor to consider. So we will keep it.

# Feature Selection

In [ ]:
#Only Extracting the int64 type features
num_features = []
for i,j in zip(train.columns,train.dtypes.tolist()):
    if(j=="object" or j == "datetime64[ns]"):
        pass
    else:
        num_features.append(i)

In [ ]:
num_features[:20]

In [ ]:
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier



model = XGBClassifier(
        n_estimators = 500,
        max_depth=11,
        learning_rate=0.06,
        subsample=0.85,
        colsample_bytree=0.6,
        random_state=0,
        tree_method='gpu_hist'
)


If you uncomment the below code It will take a lot of time. 

In [ ]:
#Selecting Top 50 features
#selector = RFE(model,n_features_to_select=50,step=2)
#selector.fit(X_train[num_features],y_train)

In [ ]:
feats = ['feature_2',
 'feature_9',
 'feature_12',
 'feature_16',
 'feature_19',
 'feature_24',
 'feature_30',
 'feature_32',
 'feature_35',
 'feature_37',
 'feature_39',
 'feature_42',
 'feature_45',
 'feature_57',
 'feature_61',
 'feature_62',
 'feature_63',
 'feature_67',
 'feature_69',
 'feature_76',
 'feature_81',
 'feature_82',
 'feature_83',
 'feature_88',
 'feature_105',
 'feature_120',
 'feature_121',
 'feature_122',
 'feature_124',
 'feature_129']

Let's Analyze the Top 50 features distribution over the span of Data

In [ ]:
for feats in feats:
    fig, axs = plt.subplots(1, 3, figsize=(15, 4))
    sns.distplot(train[feats], ax=axs[0])
    sns.distplot(train[train.weight !=0][feats], ax=axs[1])
    sns.boxplot(train[train.weight !=0][feats], ax=axs[2])
    fig.suptitle(feats, fontsize=15, y=1.1)
    
    axs[0].set_title('Distribution of feats')
    axs[1].set_title('weight ! 0')
    axs[2].set_title('Box Plot Distribution')
    
    
    plt.tight_layout()
    plt.show()

* One thing here to be noticed that Most of the features, (weight !=0) the distribution is very skewed.

In [ ]:
from sklearn.inspection import permutation_importance

def permutation_feature_selection(model,xtrain,ytrain,n_repeats=2):

    '''
    model : Empty model constructor of any ML algorithm.(e.g RandomForestRegressor(random_state=2020))
    xtrain : Training dataset
    ytrain : Output label of training dataset
    n_repeats : The number of times the feature will be shuffled

    '''


    model.fit(xtrain,ytrain)
    train_cols = []
    val_cols = []

    # Feature Selection on just Training data

    r_train = permutation_importance(model,xtrain,ytrain,n_repeats=n_repeats,random_state=0)
    for i in r_train.importances_mean.argsort()[::-1]:
        train_cols.append(xtrain.columns[i])



# Calling Code
model = XGBClassifier(
        n_estimators = 500,
        max_depth=11,
        learning_rate=0.06,
        subsample=0.85,
        colsample_bytree=0.6,
        random_state=0,
        tree_method='gpu_hist'
)

#Takes a lot of time
#t_cols= permutation_feature_selection(model,train[num_features],train['action'])

In [ ]:
#Top 50 features based on Permutation feature Importance.
#t_cols[:50]